<div >
<img src = "../banner.jpg" />
</div>

# Desbalance de Clases Extremo: Fraude

En este tutorial nos centraremos en detectar [fraude en tarjetas de crédito](https://www.kaggle.com/mlg-ulb/creditcardfraud). Los datos que utilizaremos contienen transacciones realizadas con tarjetas de crédito en septiembre de 2013 por titulares de tarjetas europeos.

Este conjunto de datos presenta transacciones que ocurrieron en dos días, donde tenemos 492 fraudes de 284,807 transacciones. El conjunto de datos está muy desequilibrado: la clase positiva (fraudes) representa el 0,172% de todas las transacciones. Esta base de datos suele utilizarse como "benchmark" para los estudios de desbalances de clases.


## Configuración Inicial

In [ ]:
#Cargar librerías 
require("pacman")

p_load("tidyverse",
       "glmnet",
        "caret")


## Datos
Los datos que vamos a usar en esta sección son  datos de detección de fraude en tarjetas de crédito que están alojados en google.

Esta base de datos solo contiene variables numéricas que son el resultado de una transformación PCA. Lamentablemente, debido a cuestiones de confidencialidad, no se difunden las características originales ni más información general sobre los datos. Las características V1, V2,… V28 son los componentes principales obtenidos con PCA, las únicas características que no se han transformado con PCA son 'Tiempo' (`Time`) y 'Cantidad' (`Amount`). La variable `Time` contiene los segundos transcurridos entre cada transacción y la primera transacción en el conjunto de datos. La variable `Amount`  es el monto de la transacción; esta variable se puede utilizar para el aprendizaje sensible a los costos, por ejemplo. La característica 'Clase' es la variable de respuesta y toma el valor 1 en caso de fraude y 0 en caso contrario.


In [ ]:
# cargar datos
fraud <- read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

# Ver las primeras filas del dataframe.
head(fraud)

In [ ]:
# Veamos cómo se distribuye la variable de interés.
fraud<- fraud %>% mutate(Fraude = factor(Class,levels=c(0,1),labels=c("Negativo","Positivo")))
table(fraud$Fraude)


In [ ]:
prop.table(table(fraud$Fraude))

In [ ]:
data<- fraud  %>% group_by(Fraude) %>% tally()

In [ ]:
data

In [ ]:
ggplot(data,aes(x = Fraude, y = n, fill = Fraude)) +
  geom_bar(stat = "identity") +
  theme_minimal() +
  scale_fill_manual(values = c("Positivo" = "orange", "Negativo" = "blue")) + # Colors can be changed
  labs(x = "", y = "count") # Customize axis labels if needed


Menos del 1% de los casos es fraude. Las dos clases: fraude y no fraude están desbalanceadas, esto implica un reto grande para nuestros modelos de predicción, pues naturalmente tenderán a predecir que los casos no son fraude, dada la poca información que hay de los fraudes. Sin embargo, lo que más nos interesa en este caso es predecir los fraudes.


## Limpiar y dividir la muestra

Los datos crudos tienen un par de problemas : las columnas  `Time` y `Amount` tiene mucha varianza. Vamos eliminar la columna `Time` (ya que no será muy útil) y vamos a tomar el logaritmo de `Amount`.


In [ ]:
# Eliminar las columnas 'Class' 'Time'
fraud<- fraud  %>% select(-Class,-Time)


# Convertir 'Amount' a  logs
fraud<- fraud  %>% mutate(Log_Amount = log(Amount + 0.001)) %>% select(-Amount)


### División de la muestra

- El objetivo es predecir bien fuera de muestra

- No queremos sobreajustar a la muestra
  
- Vamos a definir 2 bases

  - Muestra de entrenamiento: vamos a estimar los modelos, buscar parámetros, etc.
  
  -  Muestra de prueba que solo vamos a usar para evaluar los modelos


In [ ]:
# Dividir los datos en conjuntos de entrenamiento (train), validación (val) y prueba (test)
set.seed(123) # Para reproducibilidad


train_indices <- as.integer(createDataPartition(fraud$Fraude, p = 0.8, list = FALSE))
train <- fraud[train_indices, ]
test <- fraud[-train_indices, ]

dim(train)


In [ ]:
dim(test)

In [ ]:
table(train$Fraude)

In [ ]:
prop.table(table(train$Fraude))

In [ ]:
table(test$Fraude)

In [ ]:
prop.table(table(test$Fraude))

Note que la proporción de fraudes se preserva casi intacta para los conjuntos de entrenamiento, validacion y test. Esto no siempre suele ser el caso cuando las clases están tan desbalanceadas.


# Logit

If you have an imbalanced data set, first try training on the true distribution. If the model works well and generalizes, you're done! If not, try  resampling techniques.

In [ ]:
ctrl<- trainControl(method = "cv",
                     number = 5,
                     classProbs = TRUE,
                     verbose=FALSE,
                     savePredictions = T)


In [ ]:
#logit
set.seed(1410)
fraude_logit_orig <- train(Fraude~., 
                       data = train, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial")

fraude_logit_orig

In [ ]:
test<- test  %>% mutate(fraude_hat_logit_orig=predict(fraude_logit_orig,newdata = test,
                           type = "raw"))

In [ ]:
confusionMatrix(data = test$fraude_hat_logit_orig, 
                reference = test$Fraude, positive="Positivo", mode = "prec_recall")

## Remuestreo

Hay varios enfoques:

    1. Up-sampling.  Simulates or imputes additional data points of the minority class to improve balance across classes, while 
    2. Down-sampling. Randomly reduces the number of the majority class  to improve the balance across classes.
    3. Híbrido: 
        3.1 SMOTE
        3.2 ROSE

<div >
<img src = "sampling_methods.png" />
</div>

### Up Sampling

In [ ]:
set.seed(1103)
upSampledTrain <- upSample(x = train,
                           y = train$Fraude,
                           ## keep the class variable name the same:
                           yname = "Fraude")
dim(train)

dim(upSampledTrain)

table(upSampledTrain$Fraude)


In [ ]:

set.seed(1410)

fraude_logit_upsample <- train(Fraude~., 
                       data = upSampledTrain, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial")

fraude_logit_upsample



### Down Sampling

In [ ]:

set.seed(1103)
downSampledTrain <- downSample(x = train,
                           y = train$Fraude,
                           ## keep the class variable name the same:
                           yname = "Fraude")
dim(train)

dim(downSampledTrain)

table(downSampledTrain$Fraude)


In [ ]:
set.seed(1410)

fraude_logit_downsample <- train(Fraude~., 
                       data = downSampledTrain, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial")

fraude_logit_downsample


### SMOTE

<div >
<img src = "smote.png" />
</div>

In [ ]:
p_load("smotefamily")

predictors<-colnames(train  %>% select(-Fraude))
head( train[predictors])


In [ ]:
smote_output = SMOTE(X = train[predictors],
                     target = train$Fraude)
smote_data = smote_output$data


In [ ]:
table(train$Fraude)
table(smote_data$class)

In [ ]:
set.seed(1410)

fraude_logit_smote <- train(class~., 
                       data = smote_data, 
                       method = "glm",
                       trControl = ctrl,
                       family = "binomial")

fraude_logit_smote



ROSE uses smoothed bootstrapping to draw artificial samples from the feature space neighbourhood around the minority class.

In [ ]:
# p_load(ROSE)

# set.seed(9560)
# rose_train <- ROSE(Frude ~ ., data  = training)$data                         
# table(rose_train$Class) 

In [ ]:
# set.seed(1410)

# fraude_logit_rose <- train(class~., 
#                        data = rose_train, 
#                        method = "glm",
#                        trControl = ctrl,
#                        family = "binomial")

# fraude_logit_rose


# Test set Performance



In [ ]:
test<- test  %>% mutate(fraude_hat_smote_orig=predict(fraude_logit_smote,newdata = test,
                           type = "raw"))

In [ ]:
confusionMatrix(data = test$fraude_hat_logit_orig, 
                reference = test$Fraude, positive="Positivo", mode = "prec_recall")

In [ ]:
confusionMatrix(data = test$fraude_hat_smote_orig, 
                reference = test$Fraude, positive="Positivo", mode = "prec_recall")